# Content

1. Goal
2. Setup
3. Connect to PostgreSQL
4. Upload Historical Data to PostgreSQL
5. Sample of Uploaded Data

# Goal

The aim of this jupyter notebook is to upload data to a PostgreSQL database. It concerns historical data from:

- CoinGecko
- Reddit
- Google Trends

# Setup

### Install Packages

In [1]:
# If you installed Python using Anaconda or Miniconda, then use conda:
# conda install -c anaconda psycopg2
# conda install -c anaconda sqlalchemy

# If you installed Python any other way, then use pip:
# !pip install psycopg2
# !pip install SQLAlchemy

### Load Packages

In [2]:
import csv                             # Used to insert data into tables 
import sys
import psycopg2
from sqlalchemy import create_engine

# Connection to PostgreSQL

Reference to documentation: https://www.psycopg.org/docs/

In [3]:
# Connection to postgres DB
# Access is open to...
# ::/0
# 0.0.0/0
# This is set in AWS -> RDS -> 'database name' -> VPC security groups -> select any security group -> Inbound rules -> Edit inbound rules

try:
    conn = psycopg2.connect(dbname   = "datalakebisasam",
                            user     = "bisasam",
                            password = "1234asdf",
                            host     = "datalake.clnjs1yqzw0z.us-east-1.rds.amazonaws.com")
except psycopg2.Error as e:
    print(f"Error connecting to DB: {e}")

# Upload Historical Data to PostgreSQL

In [4]:
# Open a cursor to perform DB operations

cur = conn.cursor()

In [5]:
# Enabling auto-commit on existing connection
# Otherwise if commit() is not called, the effect of any data manipulation would be lost.
conn.autocommit = True

# Check if autocommit is on/off
#print(conn.autocommit)

In [6]:
# List all DB's

query = "SELECT datname FROM pg_database;"


try:
    cur.execute(query)
except psycopg2.Error as e:
    print(f"Error: {e}")


for Database in cur: 
    print(f"{Database}")

('template0',)
('rdsadmin',)
('template1',)
('postgres',)
('datalakebisasam',)


In [7]:
# Drop tables if they already exist (which is the case if this notebook was executed before)

query_A = "DROP TABLE historical_coin_data;"
query_B = "DROP TABLE historical_reddit_data;"
query_C = "DROP TABLE historical_google_data;"


try:
    cur.execute(query_A)
    cur.execute(query_B)
    cur.execute(query_C)
except psycopg2.Error as e:
    print(f"Error: {e}")

In [8]:
# Create tables

query_A = "CREATE TABLE historical_coin_data (prices DOUBLE PRECISION, market_caps DOUBLE PRECISION, total_volumes DOUBLE PRECISION, coin TEXT, date TEXT, date_unix DOUBLE PRECISION);"
query_B = "CREATE TABLE historical_reddit_data (post_title TEXT, num_comments INTEGER, subreddit TEXT, subreddit_subscribers INTEGER, date TEXT, date_unix DOUBLE PRECISION);"
query_C = "CREATE TABLE historical_google_data (date TEXT, convex_finance INTEGER, ribbon_finance INTEGER, rari_governance_token INTEGER, gmx INTEGER, nftx INTEGER, category TEXT);"


try:
    cur.execute(query_A)
    cur.execute(query_B)
    cur.execute(query_C)
except psycopg2.Error as e:
    print(f"Error: {e}")

In [9]:
# Show all tables in the DB we are connected

query = "SELECT * FROM pg_catalog.pg_tables WHERE schemaname != 'pg_catalog' AND schemaname != 'information_schema';"


try:
    cur.execute(query)
except psycopg2.Error as e:
    print(f"Error: {e}")


for Database in cur:
    print(f"{Database}")

('public', 'historical_twitter_data', 'bisasam', None, False, False, False, False)
('public', 'coin_cata_historical', 'bisasam', None, True, False, False, False)
('public', 'historical_coin_data', 'bisasam', None, False, False, False, False)
('public', 'historical_reddit_data', 'bisasam', None, False, False, False, False)
('public', 'historical_google_data', 'bisasam', None, False, False, False, False)


In [10]:
# Get column names of table

query = "SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'historical_reddit_data';"


try:
    cur.execute(query)
except psycopg2.Error as e:
    print(f"Error: {e}")


for Database in cur: 
    print(f"{Database}")

('post_title', 'text')
('num_comments', 'integer')
('subreddit', 'text')
('subreddit_subscribers', 'integer')
('date', 'text')
('date_unix', 'double precision')


In [11]:
# Insert coin data

query = "INSERT INTO historical_coin_data VALUES (%s, %s, %s, %s, %s, %s)"
file  = ".\Datasets\historical_data_coin.csv"


with open(file, "r", encoding="utf8") as f:
    reader = csv.reader(f)
    next(reader) # Skip header row
    for row in reader:
        cur.execute(query, row)
f.close()

In [12]:
# Insert reddit data

query = "INSERT INTO historical_reddit_data VALUES (%s, %s, %s, %s, %s, %s)"
file  = ".\Datasets\historical_data_reddit.csv"


with open(file, "r", encoding="utf8") as f:
    reader = csv.reader(f)
    next(reader) # Skip header row
    for row in reader:
        cur.execute(query, row)
f.close()

In [13]:
# Insert google data

query = "INSERT INTO historical_google_data VALUES (%s, %s, %s, %s, %s, %s, %s)"
file  = ".\Datasets\historical_data_google.csv"


with open(file, "r", encoding="utf8") as f:
    reader = csv.reader(f)
    next(reader) # Skip header row
    for row in reader:
        cur.execute(query, row)
f.close()

# Sample of Uploaded Data

In [14]:
# Show rows form table 

query = "SELECT * FROM historical_reddit_data LIMIT 3;"


try:
    cur.execute(query)
except psycopg2.Error as e:
    print(f"Error: {e}")

for Database in cur:
    print(f"{Database}")

("Music NFT'S", 0, 'ecomi', 22457, '31-10-2021', 1635690306.0)
('Fresh music !!', 0, 'audius', 8065, '31-10-2021', 1635689985.0)
('The Toys that Made Us', 1, 'ecomi', 22457, '31-10-2021', 1635685133.0)


In [15]:
# Terminate connection

conn.close()